In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import time

from plasma.parallel_processing import communicators, queues
from plasma.parallel_processing.communicators.tree_flow import TreeFlow
from plasma.functional import partials

/u02/thanhhm/research-idea/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(x):
    time.sleep(2)
    return x * 2

def compute2(x):
    time.sleep(2)
    return x - 1

In [4]:
agg = communicators.Aggregator(total=10, process_base=False)
class Flow(TreeFlow):

    def __init__(self):
        super().__init__()

        self.register_chain(queues.ProcessQueue(5), None, compute)
        self.register_chain(queues.ProcessQueue(5), compute, compute2)
        self.register_chain(queues.TransferQueue(), compute, partials(print, 'step 1'))
        self.register_chain(queues.TransferQueue(), compute2, partials(print, 'step 2'))

flow = Flow()
flow

<function compute at 0x7fbe205332e0>
	|--<function compute2 at 0x7fbe20533380>
		|--print(step 2,*,**)
	|--print(step 1,*,**)

In [ ]:
flow.run()

step 1 0
step 1 2
step 1 4
step 1 6
step 1 8
step 1 10
step 1 12
step 1 14
step 2 -1
step 1 16
step 1 18
step 2 1
step 2 5
step 2 3
step 2 7
step 2 9
step 2 11
step 2 13
step 2 15
step 2 17


In [6]:
for q, _ in flow.inputs():
    for i in range(10):
        q.put(i)